# Results

In the below tables summarise the results for each dataset. 


The following represent the scenarios considered for the ML reconciliation approach:

```
CASE1 - When the validation loss considers only the bottom level time series

CASE2 - When the validation loss considers the entire hierarchy

```

For each CASE1 and CASE2 3 different lambda configurations are carried out as below:

```
lambda - [0.0001, 0.1] 

Lambda1 - lambda == 1

Lambda2 - lambda [1.1, 5]
```

The median and mean across 5 different seeds are reported.


```
Example naming convention:

ML_CASE1 - This is CASE1 lambda median forecasts

ML_CASE1_MEAN - This is CASE1 lambda mean forecasts

ML_CASE1_Lambda1 - This is CASE1 Lambda1 median forecasts

ML_CASE1_Lambda1_MEAN - This is CASE1 Lambda1 mean forecasts
```


The results are also shown for full forecast horizon and half forecast horizon.
```
Prison : horizon = 8, half horizon = 4
Tourism: horizon = 12, half horizon = 6
Wikipedia: horizon = 7, half horizon = 3
Labour: horizon = 12, half horizon = 4
```

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
RESULTS_PATH = '../results/errors/'

In [3]:
def get_error_table(data_frame, LEVELS):
    errors = data_frame.groupby(by=['error_metric', 'level']).mean().reset_index().round(2)
    errors = pd.pivot(errors, values ='error', columns='error_metric', index='level')
    errors.index = LEVELS
    overall_error = data_frame.groupby(by='error_metric').mean()[['error']].transpose()
    overall_error.index = ['Overall']
    return errors.append(overall_error).round(2)

In [4]:
def get_percentage_improvement(base_df, approach_df):
    return (base_df - approach_df)/base_df*100

In [5]:
def get_results(data, model, LEVELS, FC_TYPES, column_order, short_horizon = False):
    column_names = {
        'base': 'BASE',
        'case1': 'ML_CASE1',
        'case1_mean': 'ML_CASE1_MEAN',
        'case1_lambda1': 'ML_CASE1_Lambda1',
        'case1_lambda1_mean': 'ML_CASE1_Lambda1_MEAN',
        'case1_lambda2': 'ML_CASE1_Lambda2',
        'case1_lambda2_mean': 'ML_CASE1_Lambda2_MEAN',
        'case1_one_seed': 'ML_CASE1_ONE_SEED',
        'case2': 'ML_CASE2',
        'case2_mean': 'ML_CASE2_MEAN',
        'case2_lambda1': 'ML_CASE2_Lambda1',
        'case2_lambda1_mean': 'ML_CASE2_Lambda1_MEAN',
        'case2_lambda2': 'ML_CASE2_Lambda2',
        'case2_lambda2_mean': 'ML_CASE2_Lambda2_MEAN',
        'case3': 'ML_CASE3',
        'case4': 'ML_CASE4',
        'bottomup': 'BU',
        'ols': 'OLS',
        'wls': 'WLS',
        'mintsample': 'MinT(Sample)',
        'mintshrink': 'MinT(Shrink)',
        'erm': 'ERM',
        'seeds_prev_params': 'PREV_PARAMS_SEEDS',
        'one_seed_prev_params': 'PREV_PARAMS_ONE_SEED',
        'seeds_prev_params_mean': 'PREV_PARAMS_SEEDS_MEAN'
    }
    
    error_dic = {}
    error_tables = {}
    error_percentages = {}
    final_percentage_tables = {}
    
    for fc_type in FC_TYPE:
        file_name = f'{data}_{model}'
        if short_horizon:
            file_name = f'{file_name}_short_horizon'
        error_dic[column_names[fc_type]] = pd.read_csv(f'{RESULTS_PATH}{file_name}_{fc_type}_errors.csv', index_col=0)
        
    for fc_name, error_df in error_dic.items():
        error_tables[fc_name] = get_error_table(error_df, LEVELS)
        
    for fc_name, error_df in error_tables.items():
        error_percentages[fc_name] = get_percentage_improvement(error_tables['BASE'], error_df)
        
    for error_metric in ['MSE', 'MAE', 'SMAPE']:
        dataframe_error = []
        for fc_name, error_df in error_percentages.items():
            if fc_name =='BASE':
                continue
            error_metric_percentages = error_df[[error_metric]]
            error_metric_percentages.columns = [fc_name]
            dataframe_error.append(error_metric_percentages)
        final_percentage_tables[error_metric] = pd.concat(dataframe_error, axis=1)[column_order]
        
    return error_tables['BASE'], final_percentage_tables

## New Results

## Prison

In [6]:
LEVELS_prison = ['Australia', 'State', 'Gender', 'Legal', 'Indigenous']
data_prison = 'prison'
FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintshrink', 'erm',
               'case1', 'case1_mean', 'case1_lambda1', 'case1_lambda1_mean', 'case1_lambda2', 'case1_lambda2_mean',
               'case2', 'case2_mean', 'case2_lambda1', 'case2_lambda1_mean', 'case2_lambda2', 'case2_lambda2_mean']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Shrink)', 'ERM', 'ML_CASE1', 'ML_CASE1_MEAN',
                 'ML_CASE1_Lambda1','ML_CASE1_Lambda1_MEAN','ML_CASE1_Lambda2','ML_CASE1_Lambda2_MEAN',
                  'ML_CASE2','ML_CASE2_MEAN','ML_CASE2_Lambda1','ML_CASE2_Lambda1_MEAN',
                  'ML_CASE2_Lambda2', 'ML_CASE2_Lambda2_MEAN']

model_arima ='arima'
base_errors_prison_arima, percentages_prison_arima = get_results(data_prison, model_arima, LEVELS_prison, FC_TYPE, column_order_1)



CASE 1: Only bottom level validation loss
CASE 2: Whole hierarchy validation loss

Lambda [0.0001, 0.1] 

Lambda1 -- Lambda =1

Lambda2 -- Lambda [1.1, 5]

ARIMA

In [7]:
percentages_prison_arima['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-1204.936229,-13.275846,-230.437789,-114.450766,-150.126572,-902.877381,-921.621992,-1187.037023,-1154.926767,-203.576973,-172.687790,-902.877381,-921.621992,-1080.427882,-1171.742917,-965.237615,-970.969927
State,-0.741349,8.906267,14.764202,20.707253,9.486221,6.118457,4.068817,-10.476289,-9.556132,47.350912,51.846227,6.118457,4.068817,1.830759,-5.693208,3.305897,2.336424
Gender,-4.889860,10.048874,13.384368,19.009192,15.612176,2.485159,0.415169,-12.069117,-11.132581,46.191271,50.895363,2.485159,0.415169,-1.179969,-7.740838,2.224576,2.061136
Legal,-12.232326,-10.070241,-11.037498,-3.632026,-59.544452,-7.105612,-8.635438,-18.380198,-16.119184,22.247171,19.688146,-7.105612,-8.635438,-8.213585,-13.391705,-8.302539,-8.382621
Indigenous,0.000000,1.381039,1.571899,8.681385,-36.429845,3.348354,2.298387,-3.928738,-2.451234,22.040824,20.750464,3.348354,2.298387,2.558476,-0.751334,2.695662,2.479319
Overall,-63.001288,3.695151,-4.110239,7.575853,-15.393930,-42.294162,-44.937381,-69.222360,-66.448091,26.688119,30.471385,-42.294162,-44.937381,-53.867637,-64.235912,-46.738537,-47.455759


In [41]:
def draw_plot(dataframe):
    plot_df = dataframe.reset_index()
    plot_df = pd.melt(plot_df, id_vars=['index'], value_vars=plot_df.columns[1:])
    fig = px.bar(plot_df, x="index", y="value",
                 color='variable', barmode='group',
                 height=400)
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y = -0.25
    ))
    fig.show()

In [42]:
draw_plot(percentages_prison_arima['MSE'].iloc[:, 0:11])

In [43]:
draw_plot(percentages_prison_arima['MSE'].iloc[:, [0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16]])

In [38]:
# percentages_prison_arima['MSE'].iloc[:, 5:].style.highlight_max(color = 'lightgreen', axis = 1)

ETS

In [8]:
model_ets ='ets'
base_errors_prison_ets, percentages_prison_ets = get_results(data_prison, model_ets, LEVELS_prison, FC_TYPE, column_order_1)
percentages_prison_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-3671.615741,-95.390013,-1439.641421,-932.717901,-5300.929077,-1958.030614,-1981.595850,-2866.015061,-2777.025390,-2695.406140,-2602.944786,-2621.183024,-2374.934330,-2510.402668,-2141.728770,-1995.221168,-1876.807859
State,23.569975,14.275249,20.715633,20.751485,-14.648431,25.886796,22.040608,28.262217,28.678703,30.734136,30.717010,31.086332,34.127619,31.363202,33.554561,31.707028,33.080479
Gender,12.276370,3.129698,7.906455,7.916996,-34.866193,14.878043,13.873077,17.463679,17.865566,20.315925,20.539995,20.648621,24.094080,20.306030,23.024502,21.897108,22.999930
Legal,6.081990,-0.466017,-4.214015,0.125546,-24.828064,6.036338,2.840160,7.897161,8.169145,9.665087,9.240427,9.598000,9.309613,9.479742,10.485179,8.563508,9.372716
Indigenous,0.000000,-5.287354,-14.794398,-7.243976,-86.333464,0.197037,-2.656325,1.481197,1.695622,2.874767,2.517540,2.690033,2.435687,2.664046,3.436501,2.089130,2.645780
Overall,-35.948105,4.501743,-11.767894,-3.108093,-102.595354,-11.035594,-14.127090,-21.195871,-19.633368,-16.604960,-15.421260,-15.401837,-9.996316,-13.907386,-6.999108,-6.589738,-3.909893


In [41]:
# percentages_prison_ets['MSE'].iloc[:, 5:].style.highlight_max(color = 'lightgreen', axis = 1)

### Short horizon

ARIMA

In [11]:
model_arima ='arima'
base_errors_prison_arima, percentages_prison_arima = get_results(data_prison, model_arima, LEVELS_prison, FC_TYPE, column_order_1, True)


In [12]:
percentages_prison_arima['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-3404.582840,-19.775671,-471.969377,-191.504184,-1519.071403,-1836.213430,-1919.672613,-3336.420326,-3138.158949,-1714.203104,-3942.554463,-1836.213430,-1919.672613,-2726.500329,-3228.850633,-2254.661185,-2275.492901
State,12.879618,6.734478,14.029956,17.439712,9.036993,17.408518,15.087357,-2.149563,-1.271665,41.724570,27.309656,17.408518,15.087357,13.692283,4.917550,12.616933,11.161647
Gender,13.392442,10.023429,14.739694,18.102436,16.442747,18.810520,16.596383,2.475224,3.517781,43.294603,28.176441,18.810520,16.596383,15.252492,8.191159,16.633475,16.112409
Legal,-5.374847,-14.083709,-11.962387,-7.717306,-63.458886,-0.870457,-2.983930,-17.800227,-14.823122,15.010035,-9.642062,-0.870457,-2.983930,-1.918467,-9.126515,-5.234346,-5.439736
Indigenous,0.000000,-5.975455,-5.263362,-0.729488,-54.737655,3.125744,1.636931,-8.619112,-6.481494,14.020959,-2.333504,3.125744,1.636931,2.325743,-2.616495,0.326744,-0.033986
Overall,-19.319347,2.554334,3.647937,9.568925,-20.821806,-1.978749,-4.790131,-31.170117,-28.135235,20.156545,-14.403940,-1.978749,-4.790131,-12.051874,-23.573353,-8.996404,-9.974963


In [40]:
# percentages_prison_arima['MSE'].iloc[:, 5:].style.highlight_max(color = 'lightgreen', axis = 1)

ETS

In [14]:
model_ets ='ets'
base_errors_prison_ets, percentages_prison_ets = get_results(data_prison, model_ets, LEVELS_prison, FC_TYPE, column_order_1,
                                                            True)
percentages_prison_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-3033.353567,-15.322609,-1002.874308,-565.991246,-2921.571633,-641.821581,-639.992423,-1741.339062,-1626.389271,-1531.972284,-1392.616576,-1427.920894,-1079.833761,-1253.757484,-835.870255,-705.844348,-625.051447
State,25.769501,10.728614,18.853948,17.545933,15.893155,19.603229,13.689340,29.315392,29.521898,31.403147,30.709172,32.430924,34.462417,32.294343,33.385164,29.124148,30.425919
Gender,17.187082,3.108864,8.698361,7.558099,8.613978,11.124900,8.321485,21.014729,21.238537,23.285472,22.769527,24.339031,26.000879,23.915694,25.314284,21.474946,22.625923
Legal,6.290859,-7.167135,-6.374773,-4.674812,-6.987274,-1.883968,-10.531003,6.920080,7.132614,8.513593,7.146450,9.016492,6.163635,8.568799,8.719007,3.575771,4.606311
Indigenous,0.000000,-10.983168,-17.938420,-12.922318,-39.317612,-6.945100,-14.722589,0.526532,0.699836,1.811305,0.670265,1.837639,-0.860635,1.735540,1.805273,-2.074350,-1.388049
Overall,-6.138350,2.815010,-0.256886,3.008417,-18.444188,5.472439,-0.131030,6.429279,7.508467,9.971181,10.226937,11.589174,14.776460,12.638035,16.707593,13.477374,15.223780


In [39]:
# percentages_prison_ets['MSE'].iloc[:, 5:].style.highlight_max(color = 'lightgreen', axis = 1)

## Tourism

In [16]:
LEVELS_tourism = ['Australia', 'States', 'Regions']
data_tourism = 'tourism'

FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintsample', 'mintshrink', 'erm',
               'case1', 'case1_mean', 'case1_lambda1', 'case1_lambda1_mean', 'case1_lambda2', 'case1_lambda2_mean',
               'case2', 'case2_mean', 'case2_lambda1', 'case2_lambda1_mean', 'case2_lambda2', 'case2_lambda2_mean']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Sample)','MinT(Shrink)', 'ERM', 'ML_CASE1', 'ML_CASE1_MEAN',
                 'ML_CASE1_Lambda1','ML_CASE1_Lambda1_MEAN','ML_CASE1_Lambda2','ML_CASE1_Lambda2_MEAN',
                  'ML_CASE2','ML_CASE2_MEAN','ML_CASE2_Lambda1','ML_CASE2_Lambda1_MEAN',
                  'ML_CASE2_Lambda2', 'ML_CASE2_Lambda2_MEAN',]


model_arima ='arima'
base_errors_tourism_arima, percentages_tourism_arima = get_results(data_tourism, model_arima, LEVELS_tourism, FC_TYPE, column_order_1)

ARIMA

In [17]:
percentages_tourism_arima ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-336.612973,-15.402187,-186.242951,-32.443611,-102.650850,-0.952876,-102.773745,-100.625579,-2.588107,9.933037,-145.526787,-143.375279,-35.991282,-35.882361,-146.752803,-145.250560,-66.593739,-71.240518
States,-69.790932,28.159025,-13.739921,35.504637,12.530673,42.812455,14.346513,14.649768,37.068438,44.279364,-5.844932,-5.333560,28.957981,28.615112,-11.822863,-11.772848,25.943950,24.399927
Regions,0.000000,16.233769,17.287729,23.978228,20.335997,14.419214,14.572254,14.605292,18.613858,19.651973,10.721102,10.846287,17.002606,16.892998,9.323549,9.507608,15.627896,15.283159
Overall,-198.046179,3.605229,-98.842832,-1.698437,-46.294762,15.599435,-46.664407,-45.441436,13.537692,22.559528,-76.082462,-74.776077,-6.748451,-6.819672,-78.873542,-78.045812,-23.888113,-26.863459


In [38]:
# percentages_tourism_arima ['MSE'].iloc[:, 6:].style.highlight_max(color = 'lightgreen', axis = 1)

ETS

In [19]:
model_ets ='ets'
base_errors_tourism_ets, percentages_tourism_ets = get_results(data_tourism, model_ets, LEVELS_tourism, FC_TYPE, column_order_1)

In [20]:
percentages_tourism_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-74.350078,0.996985,-32.792912,26.454736,-23.578552,34.256214,-18.335489,-2.190778,-4.950822,-20.597527,53.852066,54.328059,53.659942,51.467112,14.134104,36.120810,54.117239,52.622808
States,-37.322787,2.967490,-8.087188,31.235177,-2.283716,-36.878294,0.220360,10.019350,0.350575,-8.348524,41.213899,41.980650,42.963272,40.862065,17.746094,32.378966,42.727274,41.504603
Regions,0.000000,6.104559,10.261384,7.754256,10.001538,-111.025210,7.162891,8.995216,6.332391,4.867548,13.697656,13.811089,14.894083,14.428712,9.489702,12.094508,13.985230,13.737063
Overall,-58.869128,1.931734,-23.095100,25.726605,-15.559769,4.279624,-11.710646,1.629641,-2.672845,-15.392506,47.144240,47.650606,47.522001,45.516199,14.497903,32.965267,47.691937,46.378631


In [37]:
# percentages_tourism_ets['MSE'].iloc[:, 6:].style.highlight_max(color = 'lightgreen', axis = 1)

### Short Horizon

ARIMA

In [22]:
base_errors_tourism_arima, percentages_tourism_arima = get_results(data_tourism, model_arima, LEVELS_tourism, FC_TYPE, 
                                                                   column_order_1, True)

In [23]:
percentages_tourism_arima ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-270.441142,-14.214972,-156.302629,-21.756588,-86.739568,-2.155216,-77.919881,-72.419089,1.581177,11.818582,-119.559186,-116.296745,-14.959913,-20.537647,-116.690844,-114.601786,-42.955614,-45.076463
States,-54.708017,29.896808,-7.138181,40.626618,17.024754,44.990441,23.337688,23.890463,41.929774,48.345707,0.770110,1.842440,38.932495,36.737117,-0.912410,-0.740975,34.409654,33.854381
Regions,0.000000,12.421302,14.706956,21.072981,16.879456,13.058881,12.530202,12.603748,15.590601,16.477243,8.657997,8.873477,15.121030,14.708536,8.088807,8.264066,13.372659,13.162238
Overall,-165.095281,3.085456,-85.827512,3.473199,-39.983989,14.374362,-33.871639,-30.672318,15.873930,23.570646,-64.147730,-61.999347,5.812341,2.023119,-63.166218,-61.941064,-11.191335,-12.555407


ETS

In [24]:
model_ets ='ets'
base_errors_tourism_ets, percentages_tourism_ets = get_results(data_tourism, model_ets, LEVELS_tourism, FC_TYPE, column_order_1,
                                                              True)
percentages_tourism_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-75.219439,2.487777,-29.676360,7.617090,-20.707002,78.937392,-9.462032,0.476217,-3.766515,-20.941174,51.523673,51.365423,49.407154,47.613899,7.513000,35.669695,52.196494,50.077909
States,-5.809304,7.375898,16.004130,42.280236,21.730145,27.476677,30.805816,35.913708,27.987033,19.662224,60.154336,60.003404,59.693884,58.383952,37.951200,52.979971,61.142022,59.717119
Regions,0.000000,1.256380,9.847346,-0.727899,8.877108,-85.805523,7.537024,8.515361,5.922127,4.503460,12.183010,12.005577,12.910993,12.562378,7.881863,10.713450,12.164591,11.966809
Overall,-50.104556,3.602071,-14.147982,15.557691,-6.996159,49.394179,2.438070,10.255136,5.244934,-8.109573,49.765769,49.607450,48.353697,46.827627,15.257227,37.551364,50.449029,48.698621


## Wikipedia

In [25]:
LEVELS_wiki = ['Total', 'Language', 'Access', 'Agent', 'Article']

data_wiki = 'wikipedia'
FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintshrink', 'erm',
               'case1', 'case1_mean', 'case1_lambda1', 'case1_lambda1_mean', 'case1_lambda2', 'case1_lambda2_mean',
               'case2', 'case2_mean', 'case2_lambda1', 'case2_lambda1_mean', 'case2_lambda2', 'case2_lambda2_mean']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Shrink)', 'ERM', 'ML_CASE1', 'ML_CASE1_MEAN',
                 'ML_CASE1_Lambda1','ML_CASE1_Lambda1_MEAN','ML_CASE1_Lambda2','ML_CASE1_Lambda2_MEAN',
                  'ML_CASE2','ML_CASE2_MEAN','ML_CASE2_Lambda1','ML_CASE2_Lambda1_MEAN',
                  'ML_CASE2_Lambda2', 'ML_CASE2_Lambda2_MEAN']

model_arima ='arima'
base_errors_wiki_arima, percentages_wiki_arima = get_results(data_wiki, model_arima, LEVELS_wiki, FC_TYPE, column_order_1)



ARIMA

In [26]:
percentages_wiki_arima ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total,30.844347,5.181596,55.856905,-1015.245576,-58.808207,49.211134,48.941658,43.753214,48.020864,59.601223,59.502469,69.655092,69.561335,45.011288,49.326788,51.241818,56.218813
Language,25.433718,-4.844533,37.651617,-787.849116,-86.594101,39.837009,39.606787,36.625417,40.418944,47.339274,47.369780,60.004908,60.135691,37.989515,41.839146,41.669590,45.809496
Access,32.080825,-1.230499,-129.978518,-606.638972,-93.439680,42.724063,42.548695,40.345297,43.168231,46.914236,46.910472,58.344131,58.382167,41.369591,44.225420,44.452043,47.422728
Agent,-896.607505,1.062973,-155.517605,-870.377117,-467.556773,-4491.012983,-4494.224752,-4496.642105,-4456.472731,-4443.350293,-4443.955338,-4017.260697,-4012.599400,-4474.997103,-4427.626112,-4432.211651,-4368.476816
Article,0.000000,-23.865074,-633.396724,-523.974750,-545.716574,-1490.464638,-1490.473962,-1490.504926,-1490.477909,-1490.450610,-1490.454954,-1490.385049,-1490.348108,-1490.489342,-1490.467567,-1490.464095,-1490.440613
Overall,-109.520179,-1.531834,-73.321581,-827.643424,-175.942592,-763.386431,-764.040108,-767.331960,-758.521637,-750.170135,-750.288324,-678.978224,-678.285297,-763.201697,-753.296424,-753.286906,-740.648689


ETS

In [27]:
model_ets ='ets'
base_errors_wiki_ets, percentages_wiki_ets = get_results(data_wiki, model_ets, LEVELS_wiki, FC_TYPE, column_order_1)
percentages_wiki_ets ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total,25.519796,5.902553,45.529253,-918.177460,-195.725711,36.030493,36.222535,26.557528,27.786007,28.717194,28.812484,44.701603,45.078878,52.585191,49.908474,51.316912,52.128466
Language,24.042062,4.836913,34.923472,-760.945135,-172.020681,30.319137,30.449109,24.674203,25.422918,26.066765,26.112637,36.076646,36.287792,39.663192,37.609829,40.273182,40.825614
Access,-19.433570,-16.747319,-204.655993,-773.003755,-2265.617677,-14.105532,-13.983647,-18.898840,-18.278909,-17.439580,-17.385724,-8.667327,-8.503084,-1.083176,-2.483462,-4.924566,-4.277183
Agent,-944.111416,-0.320377,-108.777567,-548.252168,-1328.311923,-4490.745352,-4489.362125,-4568.463024,-4559.574144,-4549.178649,-4548.060118,-4411.041402,-4407.607438,-3861.082993,-3844.542966,-4305.942031,-4297.456445
Article,0.000000,-26.225274,-825.784926,-530.736817,-1485.045776,-1635.668146,-1635.658758,-1635.713835,-1635.708495,-1635.708506,-1635.707044,-1635.623937,-1635.613803,-1635.563819,-1635.577502,-1635.583447,-1635.580433
Overall,-136.236013,-1.058154,-90.868802,-769.777012,-745.227696,-837.779878,-837.439045,-855.723283,-853.580996,-851.314199,-851.082164,-819.728797,-818.966376,-727.794215,-726.884091,-799.043448,-797.171133


### Short Horizon

ARIMA

In [28]:
model_arima ='arima'
base_errors_wiki_arima, percentages_wiki_arima = get_results(data_wiki, model_arima, LEVELS_wiki, FC_TYPE, 
                                                             column_order_1, True)

percentages_wiki_arima ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total,34.430066,5.866183,84.643106,-306.015140,36.706810,53.518206,53.245146,47.750191,53.109889,66.587319,65.374864,80.913462,80.617839,47.742663,52.861890,55.120517,60.980945
Language,31.022679,-5.687649,57.229309,-236.753104,38.952864,47.566566,47.308682,44.105587,49.391283,58.313438,57.213183,76.421166,76.416943,44.098052,49.173359,49.145752,54.369935
Access,39.188540,-2.158041,-61.066888,-207.485717,20.910009,52.758443,52.534104,50.033060,54.393686,59.735477,58.875108,78.419882,78.451370,50.033126,54.225571,54.566869,58.769901
Agent,-448.207170,0.726583,-28.733745,-356.574686,-194.545356,-2925.715697,-2928.106099,-2950.699208,-2904.538153,-2844.101473,-2853.674795,-2570.547485,-2570.592632,-2950.699531,-2906.441772,-2904.576774,-2856.477519
Article,0.000000,-90.694764,-2214.064776,-1712.477008,-1270.696617,-5142.804763,-5142.828468,-5143.013502,-5142.782400,-5142.546192,-5142.591085,-5141.661290,-5141.539655,-5143.013260,-5142.791979,-5142.738513,-5142.548502
Overall,-33.401502,-0.687784,0.689528,-302.958070,-19.043749,-448.651854,-449.203711,-455.926408,-445.153009,-427.820606,-430.092229,-375.995233,-376.122448,-455.931604,-445.606998,-444.330648,-433.114205


ETS

In [29]:
model_ets ='ets'
base_errors_wiki_ets, percentages_wiki_ets = get_results(data_wiki, model_ets, LEVELS_wiki, FC_TYPE, 
                                                         column_order_1, True)
percentages_wiki_ets ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total,26.404619,5.910642,85.715717,-251.133556,-58.930852,38.227349,38.443728,27.496807,28.823778,29.823569,29.929454,48.747595,49.267094,85.638664,86.377985,60.311464,61.454495
Language,29.741725,6.380009,70.537365,-203.953709,-41.085939,38.606496,38.775417,30.581556,31.584764,32.481915,32.560347,47.038108,47.391693,84.674981,85.135299,56.327722,57.194937
Access,-19.452671,-28.554225,-126.507773,-311.965637,-1395.161529,-10.131545,-9.932127,-18.561395,-17.518132,-16.277485,-16.192674,-0.683486,-0.337118,42.688090,43.241407,9.000763,10.073109
Agent,-537.007796,-1.972247,-11.825264,-198.868422,-716.532582,-3342.922595,-3341.472487,-3400.234771,-3393.166533,-3385.706689,-3385.199820,-3275.106781,-3272.439501,-2829.162338,-2822.999335,-3198.290779,-3190.071148
Article,0.000000,-105.110147,-3038.651520,-1865.331126,-6679.504617,-6016.422864,-6016.388999,-6016.790941,-6016.747341,-6016.726390,-6016.712549,-6016.040282,-6015.994893,-6014.372354,-6014.342308,-6015.614961,-6015.581648
Overall,-56.723104,-0.508650,-3.705108,-263.602996,-393.482510,-534.154927,-533.783216,-550.282528,-548.287974,-546.402316,-546.251225,-516.447822,-515.697202,-422.116635,-420.723042,-496.736338,-494.701394


## Labour Data

In [30]:
LEVELS_labour = ['Total Employees', 'Main Occupation', 'Employment Status', 'Gender']

data_labour = 'labour'
FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintsample', 'mintshrink', 'erm',
               'case1', 'case1_mean', 'case1_lambda1', 'case1_lambda1_mean', 'case1_lambda2', 'case1_lambda2_mean',
               'case2', 'case2_mean', 'case2_lambda1', 'case2_lambda1_mean', 'case2_lambda2', 'case2_lambda2_mean']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Sample)','MinT(Shrink)', 'ERM', 'ML_CASE1', 'ML_CASE1_MEAN',
                 'ML_CASE1_Lambda1','ML_CASE1_Lambda1_MEAN','ML_CASE1_Lambda2','ML_CASE1_Lambda2_MEAN',
                  'ML_CASE2','ML_CASE2_MEAN','ML_CASE2_Lambda1','ML_CASE2_Lambda1_MEAN',
                  'ML_CASE2_Lambda2', 'ML_CASE2_Lambda2_MEAN']

ARIMA

In [32]:
model_arima ='arima'
base_errors_labour_arima, percentages_labour_arima = get_results(data_labour, 
                                                                 model_arima, LEVELS_labour, FC_TYPE, column_order_1)

percentages_labour_arima['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total Employees,-88.715285,-5.150195,-34.669487,14.953236,-11.653076,0.478694,-27.284384,-32.214845,-50.452386,-49.504216,-88.911426,-96.134515,-153.033217,-92.141469,-85.843123,-83.805836,-88.911426,-96.134515
Main Occupation,16.785035,14.924121,17.425253,34.670086,22.939207,-88.651984,25.919143,26.788656,23.104842,22.604013,15.691658,14.636898,15.707347,21.253884,17.406547,17.697089,15.691658,14.636898
Employment Status,0.467585,-2.341733,0.894809,24.530701,8.593970,-42.506073,7.085361,7.989308,5.439675,5.813590,-1.689856,-2.608272,-2.013510,2.822264,1.046355,1.115655,-1.689856,-2.608272
Gender,0.000000,-1.890739,-1.499057,6.705767,2.799196,-50.623411,4.337216,5.214913,3.547699,3.525141,-1.494955,-2.132721,-0.652120,2.136822,0.980231,0.756706,-1.494955,-2.132721
Overall,-14.283412,2.742903,-2.144579,22.270409,7.621977,-49.473138,4.996777,4.577110,-1.687078,-1.545554,-15.528403,-17.849536,-29.810414,-12.577418,-13.100776,-12.573915,-15.528403,-17.849536


ETS

In [34]:
model_ets ='ets'
base_errors_labour_ets, percentages_labour_ets= get_results(data_labour, 
                                                            model_ets, LEVELS_labour, FC_TYPE, column_order_1)

percentages_labour_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total Employees,-148.714531,-15.294247,-85.678619,-6.873949,-45.502673,-394.551986,-144.744011,-142.317494,-112.588435,-112.651398,-159.712314,-357.056699,-126.732768,-122.218773,-142.381985,-137.032179,-353.586224,-346.716859
Main Occupation,8.405533,15.137206,11.690180,25.638451,16.544675,-96.985615,8.675795,8.735823,11.743292,11.420470,7.515076,-9.037898,12.982296,17.147994,8.888520,9.551591,-3.590871,-1.238174
Employment Status,-4.428752,2.580386,0.654435,19.388811,7.727508,-107.351082,-4.287233,-4.227608,-1.652970,-1.773656,-5.186633,-17.033275,-1.564610,4.595414,-4.456050,-4.036522,-15.352289,-13.269015
Gender,0.000000,4.562546,1.492789,11.182820,5.464205,-84.439830,0.089792,0.104757,1.962325,1.818284,-0.491984,-9.708738,1.743457,5.900069,-0.187066,0.028060,-8.187889,-6.665170
Overall,-29.797076,3.382068,-13.761202,14.298780,-1.037650,-160.130414,-28.812065,-28.260669,-19.937754,-20.118043,-32.713287,-84.858335,-22.537473,-17.787443,-28.339628,-26.836121,-81.572991,-78.497546


### Short horizon

ARIMA

In [35]:
model_arima ='arima'
base_errors_labour_arima, percentages_labour_arima = get_results(data_labour, 
                                                                 model_arima, LEVELS_labour, FC_TYPE, column_order_1,
                                                                True)

percentages_labour_arima['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total Employees,54.388774,18.795075,50.141146,27.000877,34.077509,67.837862,54.932413,57.953171,59.511252,56.064285,54.468908,56.965621,48.875222,63.856851,57.679122,57.952090,54.468908,56.965621
Main Occupation,27.903682,8.149990,15.944961,4.867171,16.486309,-166.857590,28.857381,30.705241,31.301716,31.511750,25.757033,25.929581,40.603862,37.845302,25.953285,27.591663,25.757033,25.929581
Employment Status,10.929656,-2.106091,2.086736,-12.716385,1.981492,-83.041190,8.900986,9.897780,12.826468,13.941813,7.324744,7.704591,19.102401,16.201536,8.835662,10.265530,7.324744,7.704591
Gender,0.000000,-8.152273,-10.488410,-34.612359,-13.233069,-85.834897,-4.856866,-3.576832,0.438782,1.174121,-2.345216,-2.224172,5.934152,3.625250,-0.829147,-0.090783,-2.345216,-2.224172
Overall,20.824709,3.133495,11.355645,-5.770717,8.262051,-85.872466,19.504227,21.165116,23.382129,23.383376,18.570289,19.167560,27.701439,27.877254,19.939400,21.089055,18.570289,19.167560


ETS

In [36]:
model_ets ='ets'
base_errors_labour_ets, percentages_labour_ets= get_results(data_labour, 
                                                            model_ets, LEVELS_labour, FC_TYPE, column_order_1,
                                                           True)

percentages_labour_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total Employees,61.946867,23.221948,66.756058,67.246988,54.053489,-15.766687,62.649755,62.981072,64.570639,65.068376,59.681054,-53.078436,75.381665,77.348907,63.028134,63.727032,-39.911931,-35.577004
Main Occupation,17.491130,1.699814,8.599841,-19.028022,6.994762,-173.911443,17.138737,16.975815,12.576935,12.670464,16.327750,8.410369,15.957858,16.158191,17.048828,16.612561,9.962347,12.718134
Employment Status,6.179319,-2.929149,0.682003,-11.338297,-0.166505,-189.265115,5.912911,5.856966,1.815566,1.274759,4.915216,2.217841,5.441370,7.840369,5.553261,5.326815,-5.527953,-2.863879
Gender,0.000000,-5.383283,-6.831418,-6.242720,-8.695105,-149.013065,-0.160554,-0.220368,-2.436644,-2.871085,-0.749252,-3.645522,-2.383126,-0.859436,-0.897214,-0.768141,-7.435857,-5.761058
Overall,18.971912,2.969629,14.141401,2.754364,10.531162,-141.692818,18.892808,18.878543,16.243484,16.118993,17.663459,-7.989470,20.362061,21.811863,18.669762,18.642529,-7.962238,-5.175455
